In [1]:
import pandas as pd
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import torch.optim as optim
from torch.utils.data import Dataset
#from torchvision import Datasets
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

In [3]:
data=pd.read_csv('engfrench.csv')

In [4]:
print(len(data))

175621


In [5]:
data.iloc[170000]

English words/sentences    The guards found a hacksaw blade in the prison...
French words/sentences     Les gardiens trouvèrent une lame de scie à mét...
Name: 170000, dtype: object

In [6]:
from transformers import AutoTokenizer

In [7]:
tokenizer = AutoTokenizer.from_pretrained("Cohere/multilingual-22-12")

tokenizer_config.json:   0%|          | 0.00/428 [00:00<?, ?B/s]

C:\Users\maddo\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\maddo\.cache\huggingface\hub\models--Cohere--multilingual-22-12. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/10.6M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [8]:
PAD_TOKEN_ID=0

In [9]:
tokenizer.add_special_tokens({'additional_special_tokens': [' _ENFR_ ']})

1

In [10]:
txten=data.iloc[0,0]
txtfr=data.iloc[0,1]


In [11]:
txten

'Hi.'

In [12]:
txtfr

'Salut!'

In [13]:
token=tokenizer.tokenize(txten)

In [14]:
token

['Hi', '.']

In [15]:
tokenfr=tokenizer.tokenize(txtfr)

In [16]:
tokenfr

['Salut', '!']

In [17]:
len(token)

2

In [18]:
len(tokenfr)

2

In [19]:

dat=txten+' _ENFR_ '+txtfr

In [20]:
dat

'Hi. _ENFR_ Salut!'

In [21]:
tokdat=(tokenizer.tokenize(dat))


In [22]:
print(tokdat)

['Hi', '.', ' _ENFR_ ', 'Salut', '!']


In [23]:
tokdat=tokenizer.convert_tokens_to_ids(tokdat)

In [24]:
for i in range(len(tokdat)):
    print(tokdat[:i],tokdat[i])

[] 22293
[22293] 119
[22293, 119] 501153
[22293, 119, 501153] 83440
[22293, 119, 501153, 83440] 106


In [44]:
class dataset(Dataset):
    def __init__(self,x):
        self.x=x
        
    def __len__(self):
        return len(self.x)
    def __getitem__(self,i):
        inp=self.x
        return torch.tensor(tokenizer.convert_tokens_to_ids(tokenizer.tokenize(inp.iloc[i,0]))),torch.tensor(tokenizer.convert_tokens_to_ids(tokenizer.tokenize(inp.iloc[i,1])))
        



In [45]:
datanewdataset=dataset(data)

In [46]:
tokenizer.decode(datanewdataset[2000][1])

"Ne t ' en va pas!"

In [47]:
tokenizer.decode(datanewdataset[2000][0])

"Don ' t leave!"

In [48]:
def custom_collate_fn(batch):
    # 'batch' is a list of tuples: [(input_tensor_1, target_tensor_1), (input_tensor_2, target_tensor_2), ...]

    # 1. Separate inputs and targets
    inputs = [torch.tensor(item[0]) for item in batch]
    targets = [torch.tensor(item[1]) for item in batch]

    # 2. Pad the inputs and targets to the length of the longest sequence in the batch
    # batch_first=True makes the output shape (BatchSize, SequenceLength)
    inputs_padded = pad_sequence(inputs, batch_first=True, padding_value=PAD_TOKEN_ID)
    targets_padded = pad_sequence(targets, batch_first=True, padding_value=PAD_TOKEN_ID)
    
    return inputs_padded, targets_padded

In [49]:
haha=DataLoader(datanewdataset,batch_size=32,shuffle=True,collate_fn=custom_collate_fn)

In [50]:
embedsize=len(tokenizer.get_vocab())

In [51]:
print(embedsize)

501154


In [52]:
for i,x in enumerate(haha):
    print(tokenizer.decode(x[1][0]))
    print(tokenizer.decode(x[0][0]))
    break

Y a - t - il un endroit en Europe que tu veuilles visiter? [PAD]
Is there somewhere in Europe you ' d like to visit?


C:\Users\maddo\AppData\Local\Temp\ipykernel_24132\1408414250.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = [torch.tensor(item[0]) for item in batch]
C:\Users\maddo\AppData\Local\Temp\ipykernel_24132\1408414250.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets = [torch.tensor(item[1]) for item in batch]


In [67]:
xa=torch.randn(10,100)

In [68]:
ha=nn.LSTM(100,200)

In [70]:
ya=ha(xa)

In [74]:
ya[0].shape

torch.Size([10, 200])

In [78]:
ya[1][0].shape

torch.Size([1, 200])

In [79]:
ya[1][1].shape

torch.Size([1, 200])

In [63]:
class enc(nn.Module):
    def __init__(self):
        super().__init__()
        self.emb=nn.Embedding(501160,10)
        self.fc1=nn.Linear(10,100)
        self.fc2=nn.Linear(100,200)
        self.fc3=nn.Linear(200,1000)
        self.fc4=nn.Linear(1000,5)
        self.rel=nn.ReLU()
    def forward(self,x):
        embx=self.emb(x)
        embx=self.rel(self.fc1(embx))
        out=self.rel(self.fc2(embx))
        out=self.rel(self.fc3(out))
        out=self.fc4(out)
        return out
        

        
        

In [64]:
encoder=enc()

In [65]:
for i,x in enumerate(haha):
    print(type(x[0]))
    ha=encoder(x[0])
    break

<class 'torch.Tensor'>


C:\Users\maddo\AppData\Local\Temp\ipykernel_24132\1408414250.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = [torch.tensor(item[0]) for item in batch]
C:\Users\maddo\AppData\Local\Temp\ipykernel_24132\1408414250.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets = [torch.tensor(item[1]) for item in batch]
